In [ ]:
from openai import OpenAI
import os
from getpass import getpass
import base64
from io import BytesIO
import json

from typing import List
from pydantic import BaseModel, confloat

In [ ]:
# input api key
# get from https://openai.com/api/
api_key = getpass("Enter your OpenAI API key: ")

#gpt api
gpt_client = OpenAI(api_key=api_key)

In [ ]:
# convert image to base64 to pass into openai api
def image_to_base64(img):
    """Convert a PIL Image to a base64 data URI for OpenAI API."""
    buffer = BytesIO()
    img.save(buffer, format="JPEG")  # or "PNG" if you prefer
    b64 = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return f"data:image/jpeg;base64,{b64}"

In [ ]:
# huskyeats api helpers
import requests

def get_nutrition_info(item_id):
    url = "https://husky-eats.onrender.com/api/menuitem/" + str(item_id)
    r = requests.get(url)
    r.raise_for_status()
    return r.json()

In [ ]:
# Pydantic Output Classes
class ClassifiedItem(BaseModel):
    id: int
    name: str
    confidence: confloat(ge=0.0, le=1.0)

class ClassificationResult(BaseModel):
    items: List[ClassifiedItem]   # [{id,name,confidence}, ...]
    explanation: str

In [ ]:
# gpt model 2, portion estimation
def gpt_portion_estimation(pil_img, classification_result, client, model="gpt-5-mini"):
    image_url = image_to_base64(pil_img)

    for item in classification_result["items"]:
      item["nutrition"] = get_nutrition_info(item["id"])

    payload = json.dumps(classification_result, indent=2)

    prompt_text = (
        "You are a nutrition analyst estimating portion sizes.\n\n"
        "INPUTS:\n"
        "1) An image of a single plate.\n"
        "2) A JSON of detected items with nutrition info, including serving size.\n\n"
        "TASK:\n"
        "- For each detected item, output ONLY the number of SERVINGS on the plate, as a decimal if needed.\n"
        "- If serving size uses EACH and shows a number N (e.g., '4 EACH'), then 1 serving = N pieces. "
        "  If you estimate P pieces on the plate, report servings = P / N.\n"
        "- For non-EACH units (g/oz/cup), estimate servings by dividing the visible amount by the serving size.\n"
        "- Do not output the piece count (P). Do not output words like 'tenders' or any free text other than the explanation field.\n"
        "- Do not invent items. Only return items present in the provided JSON.\n"
        "- Include the count in explanation for EACH type items.\n"
        "- If unsure, return your best conservative estimate.\n\n"
        "FORMAT (strict JSON):\n"
        "{\n"
        '  "servings": [ { "id": <int>, "name": "<str>", "estimated_servings": <float> }, ... ],\n'
        '  "explanation": "<1-2 short sentences>"\n'
        "}\n"
    )

    response = client.responses.parse(
        model=model,
        input=[
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": prompt_text},
                    {"type": "input_text", "text": payload},
                    {"type": "input_image", "image_url": image_url}
                ]
            }
        ],
        text_format=PortionEstimationResult
    )

    return response.output_parsed.model_dump()

In [ ]:
# output
classification_result = gpt_item_classification(pil_image, items_ranked, gpt_client)
portion_estimates = gpt_portion_estimation(pil_image, classification_result, gpt_client)